In [1]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import numpy as np
import os

def analyze_training_convergence(log_file_path, reward_column='rollout/ep_rew_mean', convergence_percentage=0.3):
    """
    加载单个DRL训练日志文件，并对训练奖励的收敛性进行量化分析。

    参数:
    - log_file_path (str): 训练日志CSV文件的路径。
    - reward_column (str): 包含平均回合奖励的列名。
    - convergence_percentage (float): 定义为“收敛阶段”的训练过程末尾的百分比。
    """
    try:
        # 1. 加载数据
        log_df = pd.read_csv(log_file_path)
        
        # 确保奖励列存在
        if reward_column not in log_df.columns:
            print(f"错误: 在文件 '{os.path.basename(log_file_path)}' 中未找到指定的奖励列 '{reward_column}'。")
            print(f"可用的列有: {log_df.columns.tolist()}")
            return

        # 移除奖励列中的任何NaN值
        reward_series = log_df[reward_column].dropna()
        
        # 2. 定义“收敛阶段”
        total_points = len(reward_series)
        convergence_start_index = int(total_points * (1 - convergence_percentage))
        
        convergence_phase_rewards = reward_series.iloc[convergence_start_index:]
        
        print(f"\n总数据点数: {total_points}")
        print(f"分析最后 {convergence_percentage*100:.0f}% 的数据作为“收敛阶段”。")
        print(f"收敛阶段从索引 {convergence_start_index} 开始，包含 {len(convergence_phase_rewards)} 个数据点。")

        # 3. 计算收敛阶段的均值和标准差
        mean_reward_convergence = np.mean(convergence_phase_rewards)
        std_dev_reward_convergence = np.std(convergence_phase_rewards)

        print("\n--- 方法一：后期训练阶段的奖励统计特性 ---")
        print(f"收敛阶段的平均回合奖励 (Mean): {mean_reward_convergence:.4f}")
        print(f"收敛阶段的奖励标准差 (Standard Deviation): {std_dev_reward_convergence:.4f}")

        # 4. 进行时间序列的平稳性检验 (ADF Test)
        print("\n--- 方法三：时间序列的平稳性检验 (ADF Test) ---")
        # ADF检验的原假设(H0)是：序列存在单位根，即非平稳。
        # 如果p值很小（例如<0.05），我们就可以拒绝原假设，认为序列是平稳的。
        adf_result = adfuller(convergence_phase_rewards)
        
        print(f"ADF 统计量 (ADF Statistic): {adf_result[0]:.4f}")
        print(f"P-值 (p-value): {adf_result[1]:.4f}")
        print("临界值 (Critical Values):")
        for key, value in adf_result[4].items():
            print(f"\t{key}: {value:.4f}")
            
        # 解读ADF检验结果
        print("\nADF检验结果解读:")
        if adf_result[1] < 0.05:
            print(f"p值 ({adf_result[1]:.4f}) 小于 0.05，我们拒绝原假设。")
            print("结论: 奖励序列在收敛阶段是平稳的，这为模型训练已收敛提供了强有力的统计证据。")
        else:
            print(f"p值 ({adf_result[1]:.4f}) 不小于 0.05，我们无法拒绝原假设。")
            print("结论: 根据ADF检验，我们没有足够的统计证据表明奖励序列是平稳的。")

    except FileNotFoundError:
        print(f"错误: 文件未找到 '{log_file_path}'。请确保文件路径正确。")
    except Exception as e:
        print(f"处理文件 '{os.path.basename(log_file_path)}' 时发生错误: {e}")

if __name__ == '__main__':
    # --- 批量处理所有训练日志文件 ---
    
    # 1. 定义存放所有训练日志的目录
    # **重要**: 请确保您的6个CSV文件都存放在一个名为 'trained_results' 的文件夹中，
    # 并且这个文件夹与本Python脚本位于同一目录下。
    # 如果目录名不同，请修改下面的变量。
    log_directory = 'CSI500trained_results'

    # 2. 定义您要分析的文件列表
    log_files_to_analyze = [
        'CSI500_seed1_trained_log.csv',
        'CSI500_seed2_trained_log.csv',
        'CSI500_seed3_trained_log.csv',
        'CSI500_seed4_trained_log.csv',
        'CSI500_seed5_trained_log.csv',
        'CSI500_seed6_trained_log.csv'
    ]

    # 3. 检查目录是否存在
    if not os.path.isdir(log_directory):
        print(f"错误: 目录 '{log_directory}' 不存在。请创建该目录并将您的训练日志CSV文件放入其中。")
    else:
        # 4. 遍历并分析每个文件
        for filename in log_files_to_analyze:
            file_path = os.path.join(log_directory, filename)
            
            print("="*60)
            print(f"正在分析文件: {filename}")
            print("="*60)
            
            if os.path.exists(file_path):
                analyze_training_convergence(file_path)
            else:
                print(f"警告: 文件 '{file_path}' 未找到，已跳过。")
            
            print("\n")



正在分析文件: CSI500_seed1_trained_log.csv

总数据点数: 2930
分析最后 30% 的数据作为“收敛阶段”。
收敛阶段从索引 2051 开始，包含 879 个数据点。

--- 方法一：后期训练阶段的奖励统计特性 ---
收敛阶段的平均回合奖励 (Mean): -0.3425
收敛阶段的奖励标准差 (Standard Deviation): 0.2143

--- 方法三：时间序列的平稳性检验 (ADF Test) ---
ADF 统计量 (ADF Statistic): -6.0469
P-值 (p-value): 0.0000
临界值 (Critical Values):
	1%: -3.4380
	5%: -2.8649
	10%: -2.5686

ADF检验结果解读:
p值 (0.0000) 小于 0.05，我们拒绝原假设。
结论: 奖励序列在收敛阶段是平稳的，这为模型训练已收敛提供了强有力的统计证据。


正在分析文件: CSI500_seed2_trained_log.csv

总数据点数: 2930
分析最后 30% 的数据作为“收敛阶段”。
收敛阶段从索引 2051 开始，包含 879 个数据点。

--- 方法一：后期训练阶段的奖励统计特性 ---
收敛阶段的平均回合奖励 (Mean): -0.2950
收敛阶段的奖励标准差 (Standard Deviation): 0.2246

--- 方法三：时间序列的平稳性检验 (ADF Test) ---
ADF 统计量 (ADF Statistic): -6.1397
P-值 (p-value): 0.0000
临界值 (Critical Values):
	1%: -3.4380
	5%: -2.8649
	10%: -2.5686

ADF检验结果解读:
p值 (0.0000) 小于 0.05，我们拒绝原假设。
结论: 奖励序列在收敛阶段是平稳的，这为模型训练已收敛提供了强有力的统计证据。


正在分析文件: CSI500_seed3_trained_log.csv

总数据点数: 2930
分析最后 30% 的数据作为“收敛阶段”。
收敛阶段从索引 2051 开始，包含 879 个数据点。

--- 方法一：后期训练阶段的奖励统计特性 ---
收敛阶段的平均回